#### Optimal Opponent Experiments
Author: Yemi Kelani

##### Google Drive Setup (Skip if running locally)

> To run this notebook, follow these steps:
> 1. Download the latest version of the [repository](https://github.com/yemi-kelani/artificial-intelligence/tree/master).
> 2. Upload the repsitory files to your Google Drive account under the path `Projects/artificial-intelligence`.
> 3. Open this file (`train.ipynb`) from your Google Drive and run the experiments.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ROOT_FOLDER = "drive/MyDrive/Projects/artificial-intelligence/models/ReinforcementLearning/"
PROJECT_PATH = f"{ROOT_FOLDER}/DeepQ_TicTacToe_v2"
NOTEBOOK_LOCATION = f"{PROJECT_PATH}/experiments"

In [ ]:
!cp {PROJECT_PATH}/DeepQAgent.py .
!cp {PROJECT_PATH}/TicTacToeGame.py .
!cp {ROOT_FOLDER}/Utils.py .

from DeepQAgent import DeepQAgent
from TicTacToeGame import TicTacToeGame, OPPONENT_LEVEL
from Utils import (
    train_agent,
    test_agent
)
MODEL_PATH = "drive/MyDrive/Projects/artificial-intelligence/trained_models/ReinforcementLearning/TicTacToeV2"

##### Local Setup (Skip if running remotely)

> 1. Run the following cells

In [ ]:
from models.ReinforcementLearning.DeepQ_TicTacToe_v2.DeepQAgent import DeepQAgent
from models.ReinforcementLearning.DeepQ_TicTacToe_v2.TicTacToeGame import TicTacToeGame, OPPONENT_LEVEL
from models.ReinforcementLearning.Utils import train_agent, test_agent
MODEL_PATH = "../../../../trained_models/ReinforcementLearning/TicTacToeV2"

##### Experiments

In [ ]:
import os
import torch

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

# DeepQ parameters

# LEARNING_RATE      = 0.0005
# MOMENTUM          = 0.90
# NUM_EPISODES       = 100000
# EPSILON            = 1.0
# GAMMA              = 0.95
# STATE_SPACE        = 9
# ACTION_SPACE       = 9
# HIDDEN_SIZE        = 128
# DROPOUT            = 0.15
# TRAIN_START        = 500
# BATCH_SIZE         = 128
# MEMORY_MAX_LEN     = 2000
# USE_TARGET_NETWORK = True
# NETWORK_SYNC_RATE  = 3

LEARNING_RATE = 0.001
MOMENTUM = 0.90
NUM_EPISODES = 100000
EPSILON = 1.0
GAMMA = 0.99
STATE_SPACE = 9
ACTION_SPACE = 9
HIDDEN_SIZE = 256
DROPOUT = 0.1
TRAIN_START = 1000
BATCH_SIZE = 64
MEMORY_MAX_LEN = 10000
USE_TARGET_NETWORK = True
NETWORK_SYNC_RATE = 100 

# model save-path roots
BASELINE = "TicTacToe-v2-BASELINE"
NAIVE    = "TicTacToe-v2-NAIVE"
AGENT    = "TicTacToe-v2-AGENT"
OPTIMAL  = "TicTacToe-v2-OPTIMAL"
SELF     = "TicTacToe-v2-SELF"

def get_full_model_path(agent_name: str = None):
  if agent_name is None:
    return os.path.join(MODEL_PATH, "DQN_checkpoint.pt") 
  return os.path.join(MODEL_PATH, agent_name + ".pt")

def supply_model(
  load_if_exists: bool = True, 
  agent_name: str = None,
  optimizer_type = ""
  ):

  agent = DeepQAgent(
      device             = DEVICE,
      epsilon            = EPSILON,
      gamma              = GAMMA,
      state_space        = STATE_SPACE,
      action_space       = ACTION_SPACE,
      hidden_size        = HIDDEN_SIZE,
      dropout            = DROPOUT,
      train_start        = TRAIN_START,
      batch_size         = BATCH_SIZE,
      memory_max_len     = MEMORY_MAX_LEN,
      use_target_network = USE_TARGET_NETWORK,
      network_sync_rate  = NETWORK_SYNC_RATE
  )

  full_model_path = get_full_model_path(agent_name)
  if load_if_exists and os.path.exists(full_model_path):
    if not full_model_path.endswith("DQN_checkpoint.pt"): 
      weights_only = True
      # weights_only = not full_model_path.endswith("DQN_checkpoint.pt")
      print("Loading Model Parameters...")
      agent.load_model(filepath=full_model_path, weights_only=weights_only)

  match optimizer_type.upper():
    case "SGD":
      optimizer = torch.optim.SGD(
        agent.parameters(), 
        lr=LEARNING_RATE, 
        momentum=MOMENTUM
      )
    case "RMS":
      optimizer = torch.optim.RMSprop(agent.parameters(), lr=LEARNING_RATE)
    case "ADAM":
      optimizer = torch.optim.Adam(agent.parameters(), lr=LEARNING_RATE)
    case _:
      print(f"Unknown optimizer '{optimizer_type}', defaulting to Adam.")
      optimizer = torch.optim.Adam(agent.parameters(), lr=LEARNING_RATE)
      
  # criterion = torch.nn.SmoothL1Loss() # Huber Loss
  criterion = torch.nn.MSELoss()
  
  return agent, optimizer, criterion

def compare_to_naive(agent_name: str, num_episodes: int = 10000):
  _agent, _, _ = supply_model(load_if_exists=True, agent_name=agent_name)
  _environment = TicTacToeGame(DEVICE, None, OPPONENT_LEVEL.NAIVE, start_as_X=True)
  test_agent(_agent, _environment, num_episodes)

def compare_to_optimal(agent_name: str, num_episodes: int = 100):
  _agent, _, _ = supply_model(load_if_exists=True, agent_name=agent_name)
  _environment = TicTacToeGame(DEVICE, None, OPPONENT_LEVEL.OPTIMAL, start_as_X=True)
  test_agent(_agent, _environment, num_episodes)

def compare_to_model(agent_name: str, model_name: str, num_episodes: int = 10000):
  _agent, _, _ = supply_model(load_if_exists=True, agent_name=agent_name)
  _enemy, _, _ = supply_model(load_if_exists=True, agent_name=model_name)
  _environment = TicTacToeGame(DEVICE, _enemy, OPPONENT_LEVEL.AGENT, start_as_X=True)
  test_agent(_agent, _environment, num_episodes)


In [ ]:
agent, _, _ = supply_model()
agent.save_model(MODEL_PATH, BASELINE)

In [ ]:
compare_to_naive(BASELINE)

In [ ]:
agent, optimizer, criterion = supply_model(
    load_if_exists=True,
    agent_name=BASELINE,
    optimizer_type="ADAM"
)
agent.LOG_DETAILS = False
agent.anneal_epsilon = True
agent.prep_cosine_anneal(0.01, 1.0, NUM_EPISODES)
# # Alternative: Linear decay with warm restart
# agent.epsilon_min = 0.01
# agent.epsilon_max = 1.0
# agent.epsilon_decay_rate = 0.9999  # Slower decay
environment = TicTacToeGame(DEVICE, None, OPPONENT_LEVEL.NAIVE)
metrics_naive = train_agent(
    agent,
    environment,
    NUM_EPISODES // 2,  # 50k episodes
    optimizer,
    criterion,
    DEVICE,
    MODEL_PATH,
    model_name=NAIVE,
    save_every=5000,
)
agent.plot_loss_history()

In [ ]:
environment.set_opponent_level(OPPONENT_LEVEL.OPTIMAL)
agent.epsilon = 0.1  # Reset to lower exploration for optimal opponent
metrics_optimal = train_agent(
    agent,
    environment,
    NUM_EPISODES // 2,  # 50k episodes
    optimizer,
    criterion,
    DEVICE,
    MODEL_PATH,
    model_name=OPTIMAL,
    save_every=5000,
)

In [ ]:
agent.save_model(MODEL_PATH, "FINAL_MODEL")
agent.plot_loss_history()

In [ ]:
compare_to_naive(f"{NAIVE}-10K")
compare_to_optimal(f"{NAIVE}-10K")

In [ ]:
compare_to_naive(f"{NAIVE}-20K")
compare_to_optimal(f"{NAIVE}-20K")

In [ ]:
compare_to_naive(f"{NAIVE}-30K")
compare_to_optimal(f"{NAIVE}-30K")

In [ ]:
compare_to_naive(f"{NAIVE}-40K")
compare_to_optimal(f"{NAIVE}-40K")

In [ ]:
compare_to_naive(f"{NAIVE}-50K")
compare_to_optimal(f"{NAIVE}-50K")

In [ ]:
compare_to_naive(f"{NAIVE}-60K")
compare_to_optimal(f"{NAIVE}-60K")

In [ ]:
compare_to_naive(f"{NAIVE}-70K")
compare_to_optimal(f"{NAIVE}-70K")

In [ ]:
compare_to_naive(f"{NAIVE}-80K")
compare_to_optimal(f"{NAIVE}-80K")

In [ ]:
compare_to_naive(f"{OPTIMAL}-5K")
compare_to_optimal(f"{NAIVE}-5K")

In [ ]:
compare_to_naive(f"{OPTIMAL}-10K")
compare_to_optimal(f"{NAIVE}-10K")

In [ ]:
compare_to_naive(f"{OPTIMAL}-15K")
compare_to_optimal(f"{NAIVE}-15K")

In [ ]:
compare_to_naive(f"{OPTIMAL}-20K")
compare_to_optimal(f"{NAIVE}-20K")

In [ ]:
compare_to_naive(f"{OPTIMAL}-25K")
compare_to_optimal(f"{NAIVE}-25K")